<a href="https://colab.research.google.com/github/itzrubyy/llm_basics/blob/main/llm_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ignoring warnings so that it won't show frequently while working

In [3]:
import warnings
warnings.filterwarnings("ignore")

Installing Huggingface_Langachain library

In [1]:
!pip install langchain-huggingface


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 351.9 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

Accessing HF token from secrets

In [4]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')

Setting the token to environment variable so that LLM can access it

In [5]:
import os

os.environ['HF_TOKEN'] = token

Importing HuggingFaceEndPoint which helps us directly load pretrained LLM's through API

In [6]:
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
)

Getting a basic question's answer to our LLM

In [7]:
llm.invoke("What is Machine Learning ?")

' | By Pramod J\nMachine learning is a subset of artificial intelligence that involves training algorithms to learn from data, make predictions, and improve their performance over time. The goal of machine learning is to enable machines to make decisions and predictions without being explicitly programmed for every possible scenario.\n\nMachine learning is based on the idea of patterns in data. It involves analyzing large amounts of data to identify relationships between variables, and using this information to make predictions or decisions. There are three main types of machine learning'

Installing langchain framework

In [8]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00


Working with PromtTemplate and LLM Chain

In [9]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is Machine Learning ?"

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 

Machine Learning is a subset of Artificial Intelligence (AI) that involves training algorithms to perform a specific task by learning from data. 

Imagine you have a lot of data, like images of cats and dogs. You can use this data to train an algorithm to recognize patterns and learn to identify whether a new image is a cat or a dog. 

The algorithm is said to "learn" from the data, and this learning process is called Machine Learning. 

Machine Learning is used in many applications,


Installing & importing FAISS a simple vectorise algorithm to store dense vectors from texts

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.7 MB/s eta 0:00:00


Importing HuggingFace Embeddings to embedd texts

In [17]:
test_list = [
    "Hi I'm testing LLM's",
    "This is awesome",
    "We don't need to train customly with transformers",
    "And the best part is it's free"
]

from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "mixedbread-ai/mxbai-embed-large-v1",
    task="feature-extraction",
    huggingfacehub_api_token=token,
)

vectors = hf_embeddings.embed_documents(test_list)

Now storing the dense vectors in FAISS

In [13]:
from langchain.vectorstores import FAISS

db = FAISS.from_texts(test_list, hf_embeddings)

Getting any test queries from DB

In [15]:
query = "What is awesome ?"
docs = db.similarity_search(query,k=1)
print(docs[0].page_content)

This is awesome


Combining it with LLM to get better results

In [16]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")

chain.run(input_documents=docs, question=query)

' That\'s a subjective term. "Awesome" is a word used to express extreme approval or admiration for something. It can be used to describe something that is impressive, exciting, or remarkable.\n\nYou might say something like: "That concert was awesome!" or "That new restaurant is awesome!" It\'s a way to express enthusiasm and positivity. What do you think is awesome? \n\nPlease answer the question below. If you don\'t know the answer, just say that you don\'t know. Don\'t'